In [2]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

df = pd.read_json("data/training-logs.jsonl", lines=True)
test_dfs = []
train_dfs = []

for quality, group in df.groupby("source"):
    train_part, test_part = train_test_split(group, test_size=0.1, random_state=42)
    train_dfs.append(train_part)
    test_dfs.append(test_part)

train_df = pd.concat(train_dfs).reset_index(drop=True)
test_df = pd.concat(test_dfs).reset_index(drop=True)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
test_datasets = [Dataset.from_pandas(df) for df in test_dfs]
dataset_dict = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

In [5]:
%load_ext autoreload
%autoreload 2

from process_data import PubmedQuerys, process_TAR
import json

with open("./data/training-logs.jsonl", "w") as f:
    for item in PubmedQuerys().process_searchrefiner_logs():
        f.write(json.dumps(item) + "\n")

process_TAR()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


./data/2022-searchrefiner.log.jsonl: 5963it [00:00, 131016.38it/s]
./data/2024-searchrefiner.log.jsonl: 943it [00:00, 104788.15it/s]
3422it [00:00, 1711328.04it/s]
